# MobileNet

**Pretrain Model on Imagenet-Tiny**

## Load dataset


In [ ]:
import torch
import torch.nn as nn


import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision import transforms as tfs
import torchvision.datasets as datasets
import torchvision.transforms as transforms


### Fruit data with five classes
Load data from self collected data

In [ ]:
folder_set = ImageFolder('/home/yang/dataset/datacollect/datafruit/datacollect01/')

In [ ]:
# check the data classes
folder_set.class_to_idx

In [ ]:
# get all picture names and labels
folder_set.imgs

In [ ]:
# get one of data
im, label = folder_set[0]

In [ ]:
im

In [ ]:
label

In [ ]:
# preprocess data
data_tf = tfs.ToTensor()

folder_set = ImageFolder('/home/yang/dataset/datacollect/datafruit/datacollect01/', transform=data_tf)

im, label = folder_set[0]

In [ ]:
im

In [ ]:
label

## Pretrain Model

### Get pretrain model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
            )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),
    
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )

        self.model = nn.Sequential(
            conv_bn(  3,  32, 2), 
            conv_dw( 32,  64, 1),
            conv_dw( 64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            nn.AvgPool2d(7),
        )
        self.fc = nn.Linear(1024, 1000)

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x

In [ ]:
mobilenet = Net()
#print(mobilenet)
print(type(mobilenet))

#mobilenet = nn.DataParallel(mobilenet).cuda()

#print keys in net
#for param in mobilenet.parameters():
#    print(type(param.data), param.size())
#    print(list(param.data))

#print(mobilenet.state_dict().keys())

In [ ]:
params = torch.load('model_best.pth.tar')['state_dict']
print(type(params))

In [ ]:
print(params)

In [ ]:


# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in params.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
# load params
mobilenet.load_state_dict(new_state_dict)

In [ ]:
#print(mobilenet)

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

In [ ]:
val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=workers, pin_memory=True)